In [2]:
import os
import sys
import re
import copy
import platform
import pandas as pd
from tqdm import tqdm
from selenium import webdriver

# set utils
util_path = os.pardir
if util_path not in sys.path:
    sys.path.append(os.pardir)
from utils.helper import load_config
from crawler import Crawler, Utills

Project_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
Save_PATH = os.path.join(Project_PATH, 'jobkorea', 'result')

# load config file
CONFIG_PATH = "config.yaml"
config = load_config(os.path.join(Project_PATH, 'jobkorea', CONFIG_PATH))
raw_data_before_processing = pd.read_excel(config['raw_data'], engine='openpyxl')
init_url = config['init_url']
selen_path_dict = config['selen_path']
chrome_opt_dict = config['chrome_option']



#location
location = "ubuntu"

# chrome option
chrome_opt = webdriver.ChromeOptions()
for _ in chrome_opt_dict['default']:
    chrome_opt.add_argument(_)

if platform.system() == 'Linux':
    for _ in chrome_opt_dict['headless']:
        chrome_opt.add_argument(_)
    chrome_opt.add_argument(f'user-agent={chrome_opt_dict["user_agent"]}')


if location == 'home':
    crawler = Crawler(selen_path = selen_path_dict['home'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = config['solution'],
                        platform_string = config['platform'])
elif location == 'laptop':
    crawler = Crawler(selen_path = selen_path_dict['laptop'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = config['solution'],
                        platform_string = config['platform'])
elif location == 'khrrc':
    crawler = Crawler(selen_path = selen_path_dict['khrrc'],
                        chromeOption=chrome_opt,
                        init_url=init_url, 
                        solutions = config['solution'],
                        platform_string = config['platform'])
elif location == 'ubuntu':
    crawler = Crawler(selen_path= selen_path_dict['ubuntu'],
                    chromeOption = chrome_opt,
                    init_url = init_url,
                    solutions = config['solution'],
                    platform_string = config['platform'])
else:
    raise NotImplementedError

In [3]:
crawler_utils = Utills(raw_data=raw_data_before_processing)
raw_data = crawler_utils.add_empty_column(column_ls='매출수 사원수 설립일 플랫폼입점여부 사용솔루션 채널톡사용여부'.split())
raw_data = crawler_utils.cleaning_url(data=raw_data)
raw_data.head()

,상호,쇼핑몰명,도메인명,전화번호,운영자이메일,영업형태,업소상태,매출수,사원수,설립일,플랫폼입점여부,사용솔루션,채널톡사용여부
0,쇼핑코리아,구례산수유마트 / ssymart,https://www.ssymart.com,/0505-200-9090,ixx100@naver.com,일반쇼핑몰,영업중,,,,,,
1,원더 비지터(wonder visitor),원더비지터 / wonder visitor,https://http://wondervisitor.com,070-4226-7566,wondervisitor@wondervisitor.com,일반쇼핑몰,영업중,,,,,,
2,(주)로샤,에어로샤 / AIROSHA,https://airosha.shop,070-7492-6858,web_master@rosha.co.kr,일반쇼핑몰,영업중,,,,,,
3,(주)삼성아이티브이,samsungitv/삼성아이티브이,https://www.samsungitv.co.kr,02-572-7367~8,imsanghun@naver.com,일반쇼핑몰,영업중,,,,,,
4,마인드케어클럽(MIND CARE CLUB),마인드케어클럽 / mindcareclub,https://mindcareclub.mysoho.com,070-8065-2935,key960210@gmail.com,일반쇼핑몰,영업중,,,,,,


In [4]:
raw_data.shape

(11185, 13)

# Shopping Mall

In [5]:
print(config['platform'])

['naver.com', 'coupang', '11st', 'interpark', 'gmarket']


In [4]:
for idx, shopping_mall_url in tqdm(enumerate(raw_data['도메인명'])):
    if idx < 2:
        rs = crawler.search_shopping_mall(shopping_mall_url, channel_talk='channel.io')
        raw_data.loc[idx, '플랫폼입점여부'] = rs['플랫폼입점여부']
        raw_data.loc[idx, '사용솔루션'] = rs['사용솔루션']
        raw_data.loc[idx, '채널톡사용여부'] = rs['채널톡사용여부']

    else:
        pass

raw_data.to_excel(os.path.join(Save_PATH, 'task_shopping_test.xlsx'))

1it [00:00,  1.41it/s]

> naver.com


11185it [00:07, 1470.78it/s]


In [5]:
for idx, shopping_mall_url in tqdm(enumerate(raw_data['도메인명'])):
    if idx < 5000:
        rs = crawler.search_shopping_mall(shopping_mall_url, channel_talk='channel.io')
        raw_data.loc[idx, '플랫폼입점여부'] = rs['플랫폼입점여부']
        raw_data.loc[idx, '사용솔루션'] = rs['사용솔루션']
        raw_data.loc[idx, '채널톡사용여부'] = rs['채널톡사용여부']

    else:
        pass

raw_data.to_excel(os.path.join(Save_PATH, 'task2_3_re.xlsx'))

11185it [5:33:04,  1.79s/it]


In [6]:
for idx, shopping_mall_url in tqdm(enumerate(raw_data['도메인명'])):
    if idx >= 5000:
        rs = crawler.search_channel_and_solution(shopping_mall_url, channel_talk='channel.io')
        raw_data.loc[idx, '플랫폼입점여부'] = rs['플랫폼입점여부']
        raw_data.loc[idx, '사용솔루션'] = rs['사용솔루션']
        raw_data.loc[idx, '채널톡사용여부'] = rs['채널톡사용여부']

    else:
        pass

raw_data.to_excel(os.path.join(Save_PATH, 'task2_3_re2.xlsx'))

11185it [7:59:57,  2.57s/it]
